This notebook is used to
1. Refine the classifiers to mitigate data imbalance
2. Create ensemble classifier
3. Perform feature ablation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install pickle5

import os
import pickle5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import copy
from random import sample
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, fbeta_score, make_scorer, accuracy_score, confusion_matrix, plot_confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, learning_curve, validation_curve
from sklearn.utils.class_weight import compute_class_weight

plt.style.use('ggplot')
%matplotlib inline

# suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn

### Read Data

In [ ]:
# Save paths on drive for models
path = f"drive/MyDrive/UNI/IPOTERI/data/cad/"
path_models = f"{path}models/"
suffix_old = ""
suffix = ""

In [ ]:
# Read data
df_train = pd.read_csv(f"{path}train{suffix}.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid{suffix_old}.csv", index_col=0)
df_test = pd.read_csv(f"{path}test{suffix_old}.csv", index_col=0)

# # Use only top 7 variables
# top_variables = [
#     "Hyperlipemia\nHistoty of hyperlipemia",
#     "FE",
#     "Previous CABG",
#     "Diabetes\nHistory of diabetes",
#     "Previous Myocardial Infarction",
#     "Smoke\nHistory of smoke",
#     "Documented resting \nor exertional ischemia",
#     "Survive7Y"
# ]
# df_train = df_train.loc[:, top_variables]
# df_valid = df_valid.loc[:, top_variables]
# df_test = df_test.loc[:, top_variables]

train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
feat_names = list(df_train.columns)

from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

In [ ]:
# All the numerical features that can be standarditazed
feat_names_num = ["Age", "FE", "Creatinina"]

# Preprocess only the numerical features
def get_preprocess_std_num(feat_names):
    def update_num_feats(x):
        if x in feat_names:
            return feat_names.index(x)

    num_feat_index = list(map(update_num_feats, feat_names_num))
    num_feat_index = [x for x in num_feat_index if x is not None]
    preprocess_std_num = ColumnTransformer(
                                transformers = [('stand', StandardScaler(), num_feat_index)], 
                                remainder="passthrough"
                            )
    return preprocess_std_num

preprocess_std = get_preprocess_std_num(feat_names)
preprocess_std_all = StandardScaler()

# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)

In [ ]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates[:1]:
            print("Model rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            

# Evaluate models
def evaluate(pipe, X, y, plot=False):
    y_pred = pipe.predict(X)
    print(classification_report(y, y_pred, digits=3))
    print(f"auc macro {roc_auc_score(y, pipe.predict_proba(X)[:, 1]):.3f}")

    if plot:
        plot_confusion_matrix(pipe, X, y, normalize=None, values_format = '')
        plt.grid(False)
    else:
        print("confusion matrix")
        print(confusion_matrix(y, y_pred))


# Train and evaluate
def train_and_evaluate(
    preprocess, 
    model, 
    hyperparams, 
    X_train, 
    y_train, 
    X_valid, 
    y_valid, 
    scoring="f1_macro", 
    iter=5000, 
    save=False, 
    savename=""
):
    pipe = Pipeline(steps=[
        ('preprocess', preprocess), 
        ('model', model)
    ])

    rand = RandomizedSearchCV(pipe,
                              param_distributions=hyperparams,
                              n_iter=iter,
                              scoring=scoring,
                              cv=2,
                              n_jobs=-1,    # use all processors
                              refit=True,   # refit the best model at the end
                              return_train_score=True,
                              verbose=True).fit(X_train, y_train)
    
    evaluate(rand.best_estimator_, X_train, y_train)
    evaluate(rand.best_estimator_, X_valid, y_valid)
    report(rand.cv_results_, n_top=5)

    if save:
        dump(rand.best_estimator_, f"{path_models}{savename}{suffix}.joblib")
    
    return rand.best_estimator_

### Sampling
Oversample and undersample methods to mitigate data imbalance

In [ ]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, OneSidedSelection, NeighbourhoodCleaningRule
from collections import Counter

overs = [
    ("smote", SMOTE(sampling_strategy=1.0, k_neighbors=1)),
    ("bordersmote", BorderlineSMOTE(sampling_strategy=1.0, k_neighbors=1)),
    ("svmsmote", SVMSMOTE(sampling_strategy=1.0, k_neighbors=1)), 
    # ADASYN(sampling_strategy=1.0, n_neighbors=1)
]

random_ratio = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
name_models = ["logreg_top", "rf_top", "adaboost_top"]
# name_models = ["logreg", "svc2", "knn", "rf1", "adaboost2", "nn", "gb", "xgb", "xgb2"]

for name in name_models:
    best_k = 0
    best_over = ""
    best_score = 0
    best_random = 0
    best_model = None
    best_X, best_y = None, None
    model = load(path_models+f"{name}{suffix}.joblib")

    for r in random_ratio:
        print("-------------------------------------------------")
        print(f"{'Oversample':<20}{'Score':<20}Random {r}")
        print("-------------------------------------------------")

        for name_over, over in overs:
            for k in range(2, 5):
                over.set_params(k_neighbors=k)
                under = RandomUnderSampler(sampling_strategy=r)
                # under = OneSidedSelection(n_neighbors=100, n_seeds_S=300)
                # under = NeighbourhoodCleaningRule(n_neighbors=25, threshold_cleaning=0.5)
                X_train_sample, y_train_sample = under.fit_resample(X_train, y_train)
                X_train_sample, y_train_sample = over.fit_resample(X_train_sample, y_train_sample)

                # Evaluate model with 5 runs
                scores = []
                for _ in range(5):
                    model.fit(X_train_sample, y_train_sample)
                    score = f1_score(y_valid, model.predict(X_valid), average="macro")
                    scores.append(score)
                score = np.mean(scores)

                if score > best_score:
                    best_k = k
                    best_over = name_over
                    best_score = score
                    best_random = r
                    best_model = copy.deepcopy(model)
                    best_X = X_train_sample
                    best_y = y_train_sample

            print(f'{name_over:<20}{score:.3f}')

    # Save the dataset and model
    combination = f"{suffix}_random_{best_over}_{name}"
    tmp = np.concatenate((best_X, np.expand_dims(best_y, 1)), axis=1)
    tmp = pd.DataFrame(tmp, columns=feat_names)

    # Current model's statistics
    print("\n")
    print(f"Name model:      \t{name}")
    print(f"Best rand_ratio: \t{best_random}")
    print(f"Best score:      \t{best_score}")
    print(f"Dataset size:    \t{len(best_y)}, {Counter(best_y)}")
    print(f"Combination:     \t{combination}")
    print("\n\n")

    # Evaluate the best model, save the data and the best model
    evaluate(best_model, best_X, best_y)
    evaluate(best_model, X_valid, y_valid)
    tmp.to_csv(path+f"train{combination}.csv")
    dump(best_model, path_models+f"{name}{combination}.joblib")

In [ ]:
tmp.to_csv(path+f"train{combination}.csv")
dump(best_model, path_models+f"{name}{combination}.joblib")

In [ ]:
# evaluate the best model
evaluate(best_model, best_X, best_y)
evaluate(best_model, X_valid, y_valid)
tmp.to_csv(path+f"train{combination}.csv")
dump(best_model, path_models+f"{name}{combination}.joblib")

### Ensemble
Combining the previous best models

In [ ]:
def build_ensemble(models):
    ensemble = []
    for m in models:
        ensemble.append((m, load(path_models+f"{m}.joblib")))
    
    return ensemble


def predict_ensemble(ensemble, X, y, threshold=0.5):
    y_proba = []
    for m in ensemble:
        y_proba.append(m.predict_proba(X))
    y_proba = np.mean(y_proba, axis=0)
    y_pred = y_proba[:, 1] > threshold
    
    return y_proba, y_pred


def evaluate_ensemble(ensemble, X, y, threshold=0.5, verbose=True):
    y_proba, y_pred = predict_ensemble(ensemble, X, y)
    if verbose:
        print(classification_report(y, y_pred, digits=3))
        print(f"auroc {roc_auc_score(y, y_proba[:, 1]):.3f} \n")
        print("confusion matrix")
        print(confusion_matrix(y, y_pred))
  
    return f1_score(y, y_pred)


def find_best_ensemble(ensemble, X_valid, y_valid):
    results = []
    while len(ensemble) > 1:
        tmp_res = []
        for m in ensemble:
            tmp = copy.copy(ensemble)
            tmp.remove(m)
            names = [_name for _name, _m in tmp]
            tmp = [_m for _name, _m in tmp]
            acc = evaluate_ensemble(tmp, X_valid, y_valid, verbose=False)
            results.append((names, tmp, acc))
            tmp_res.append((m, acc))

        m, _ = max(tmp_res, key=lambda item:item[1])
        ensemble.remove(m)
        # print(m)
        
    return max(results, key=lambda item:item[2])

In [ ]:
models = [
    "logreg_random_svmsmote_logreg",
    # "svc2_random_svmsmote_svc2",
    # "knn_random_svmsmote_knn",
    "rf1_random_bordersmote_rf1",
    "adaboost2_random_svmsmote_adaboost2",
    # "nn_random_svmsmote_nn",
    # "gb_random_svmsmote_gb",
    # "xgb_random_svmsmote_xgb",
]

models = [
    "logreg_top_random_svmsmote_logreg_top",
    "rf_top_random_svmsmote_rf_top",
    "adaboost_top_random_svmsmote_adaboost_top",
]

ensemble = build_ensemble(models)
# names, ensemble, _ = find_best_ensemble(ensemble, X_valid, y_valid)
names = list(map(lambda x: x[0], ensemble))
ensemble = list(map(lambda x: x[1], ensemble))
evaluate_ensemble(ensemble, X_valid, y_valid)
evaluate_ensemble(ensemble, X_test, y_test)
print("\n")
print(names)

### Predictions
Save the logit output (probabilities) for survival analysis

In [ ]:
# Ensemble predictions
train_prob, train_pred = predict_ensemble(ensemble, X_train, y_train)
valid_prob, valid_pred = predict_ensemble(ensemble, X_valid, y_valid)
test_prob, test_pred = predict_ensemble(ensemble, X_test, y_test)

print(sum(df_train["Survive7Y"] == train_pred) / len(df_train))
print(sum(df_valid["Survive7Y"] == valid_pred) / len(df_valid))
print(sum(df_test["Survive7Y"] == test_pred) / len(df_test))

df_train["ModelOutput"] = train_prob[:, 1]
df_valid["ModelOutput"] = valid_prob[:, 1]
df_test["ModelOutput"] = test_prob[:, 1]

In [ ]:
df_valid["ModelOutput"].to_csv("extra_valid_output.csv")
df_test["ModelOutput"].to_csv("extra_test_output.csv")")

### Feature Ablation

In [ ]:
from sklearn.inspection import permutation_importance


def get_univariate_ablation_results(X, y):
    """For each feature we set it to its mean value and save the resulting metrics"""

    features = df_valid.columns
    accuracy, auroc, f1_death, f1_survive, f1_macro = [], [], [], [], []
    for i, _ in enumerate(range(X_valid.shape[1])):
        X_copy = X.copy()
        X_copy[:, i] = np.mean(X_copy[:, i]) * np.ones(X_copy.shape[0])
        y_proba, y_pred = predict_ensemble(ensemble, X_copy, y)

        accuracy.append((features[i], round(accuracy_score(y, y_pred), 3)))
        auroc.append((features[i], round(roc_auc_score(y, y_proba[:, 1]), 3)))
        f1_death.append((features[i], round(f1_score(y, y_pred, pos_label=0), 3)))
        f1_survive.append((features[i], round(f1_score(y, y_pred, pos_label=1), 3)))
        f1_macro.append((features[i], round(f1_score(y, y_pred, average="macro"), 3)))
    
    return accuracy, auroc, f1_death, f1_survive, f1_macro


def get_multivariate_ablation_results(feat_clusters, X, y, skip_single=False):
    """For each cluster of features, we set each feature of a group to its mean and save the resulting metrics"""

    accuracy, auroc, f1_death, f1_survive, f1_macro = [], [], [], [], []
    for feat_cluster in feat_clusters:
        if skip_single and (len(feat_cluster)) == 1:
            continue

        # Create a copy of the dataset
        X_copy = X.copy()
        t = []
        for feat in feat_cluster:
            p = feat_names.index(feat)
            t.append(p)
            X_copy[:, p] = np.mean(X_copy[:, p]) * np.ones(X_copy.shape[0])

        # Predict
        y_proba, y_pred = predict_ensemble(ensemble, X_copy, y)
        accuracy.append(round(accuracy_score(y, y_pred), 3))
        auroc.append(round(roc_auc_score(y, y_proba[:, 1]), 3))
        f1_death.append(round(f1_score(y, y_pred, pos_label=0), 3))
        f1_survive.append(round(f1_score(y, y_pred, pos_label=1), 3))
        f1_macro.append(round(f1_score(y, y_pred, average="macro"), 3))
    
    return accuracy, auroc, f1_death, f1_survive, f1_macro

In [ ]:
# Univariate ablation
accuracy, auroc, f1_death, f1_survive, f1_macro = get_univariate_ablation_results(X_test, y_test)
p = pd.DataFrame({
    "auroc": [x[1] for x in auroc], 
    "f1_macro": [x[1] for x in f1_macro],
    }, index=feat_names[:-1])
p.to_csv("extra_ablation_uni_test.csv")

In [ ]:
# Multivariate hierarchical ablation
with open(f"{path}feat_cluster_hier.df", "rb") as f:
    df_clusters = pickle5.load(f)

accuracy, auroc, f1_death, f1_survive, f1_macro = get_multivariate_ablation_results(df_clusters, X_test, y_test)
p = pd.DataFrame({"cluster": df_clusters, "auroc": auroc, "f1_macro": f1_macro})
p.to_csv("multivariate_ablation_hier_nomeds.csv")

In [ ]:
# # Correlation matrix multivariate ablation
# path_corr = f"drive/MyDrive/UNI/IPOTERI/data/data/features_clustering/"
# df_corr = pd.read_csv(f"{path_corr}feat_cluster_0.4.csv", index_col=0)
# df_corr = df_corr.iloc[:, 0].apply(lambda x: set(x.split(",")))

# # Eliminate the subset feature clusters since they are redundant
# feat_clusters = []
# for i in range(len(df_corr)):
#     subset = False
#     for j in range(i+1, len(df_corr)):
#         if df_corr[i].issubset(df_corr[j]):
#             subset = True
#             break

#     if not subset:
#         feat_clusters.append(list(df_corr[i]))
# print(f"Number of clusters: {len(feat_clusters)}")

# # ablation results
# accuracy, auroc, f1_death, f1_survive, f1_macro = get_multivariate_ablation_results(feat_clusters, X_test, y_test, skip_single=True)
# ablation_results = [feat_clusters, accuracy, auroc, f1_survive, f1_death, f1_macro]
# p = pd.DataFrame(np.transpose(ablation_results))
# # p.to_csv("corr_ablation_nomeds.csv")
# p.head()

### Evaluate

In [ ]:
name = f"xgb2.joblib"
model = load(path_models+name)

# model.fit(X_train, y_train)
# evaluate(model, X_train, y_train)
evaluate(model, X_valid, y_valid)
evaluate(model, X_test, y_test)
X_train.shape